In [39]:
from snowexsql.db import get_db
from snowexsql.data import ImageData, SiteData
from rasterio.plot import show
from sqlalchemy.sql import func
import geoalchemy2.functions as gfunc
from geoalchemy2.types import Raster
from geoalchemy2.shape import to_shape
import geopandas as gpd
from snowexsql.conversions import raster_to_rasterio
from snowexsql.conversions import points_to_geopandas, query_to_geopandas, query_to_pandas
import matplotlib.pyplot as plt
import numpy as np
from datetime import date

# Pit Site Identifier of interest
site_id = '6C34'
site_name = 'Grand Mesa'

# Distance around the pit to collect data in meters
buffer_dist = 10

# Connect to the database we made.
db_name = 'snow:hackweek@db.snowexdata.org/snowex'
engine, session = get_db(db_name)

datasets = []

# Grab our sites details by site id
q = session.query(SiteData).filter(SiteData.site_id==site_id)
#q = session.query(SiteData).filter(SiteData.site_name==site_name)
#q = q.filter(SiteData.tree_canopy=="No Trees")
#q = q.filter(SiteData.date>=date(2020,2,1))
#q = q.filter(SiteData.date<=date(2020,3,1))
sites = q.all()

# Grab the pit location from a single layer
p = sites[0].geom

# Convert the point to a pyshapely
pit = to_shape(p)

# Convert it to a geopandas dataframe for easy plotting
df_pit = gpd.GeoSeries(pit)

In [40]:
# Create a polygon buffered by our distance centered on the pit
q = session.query(gfunc.ST_Buffer(p, buffer_dist))
buffered_pit = q.all()[0][0]

# Convert to a shapely shapefile object
circle = to_shape(buffered_pit)

# Convert to a geopandas dataframe
df_circle = gpd.GeoSeries(circle)

In [43]:
# Grab the rasters, union them and convert them as tiff when done
q = session.query(func.ST_AsTiff(func.ST_Union(ImageData.raster, type_=Raster)))

# Only grab rasters that are the bare earth DEM from USGS
q = q.filter(ImageData.type == 'insar amplitude').filter(ImageData.observers=='UAVSAR team, JPL').filter(ImageData.site_name == "Grand Mesa")
#q = q.filter(ImageData.id == 455701)

# And grab rasters touching the circle
q = q.filter(gfunc.ST_Intersects(ImageData.raster, buffered_pit))

# Execute the query
rasters = q.all()
print(rasters)
# Get the rasterio object of the raster
dataset = raster_to_rasterio(session, rasters)[0]
len(dataset.read(1)[0])
print(buffered_pit)
dataset.read(1)

[(<memory at 0x7faae96d1fc0>,)]
010300002020690000010000002100000000000000f0c6264100000000ae7c5041fbe19e9defc62641d2662483ad7c50418061437aeec626417d25150bad7c5041a9d91fa1ecc626417e5f6f9cac7c504110006324eac62641fe9f733bac7c50410d04851ce7c62641cb04dcebab7c504117d456a7e3c62641d093b7b0ab7c504171c9dce6dfc62641c1234c8cab7c504100000000dcc6264100000080ab7c50418f362319d8c62641c1234c8cab7c5041e92ba958d4c62641d093b7b0ab7c5041f3fb7ae3d0c62641cb04dcebab7c5041f0ff9cdbcdc62641fe9f733bac7c50415726e05ecbc626417e5f6f9cac7c5041809ebc85c9c626417d25150bad7c5041051e6162c8c62641d2662483ad7c504100000000c8c6264100000000ae7c5041051e6162c8c626412e99db7cae7c5041809ebc85c9c6264183daeaf4ae7c50415726e05ecbc6264182a09063af7c5041f0ff9cdbcdc6264102608cc4af7c5041f3fb7ae3d0c6264135fb2314b07c5041e92ba958d4c62641306c484fb07c50418f362319d8c626413fdcb373b07c504100000000dcc6264100000080b07c504171c9dce6dfc626413fdcb373b07c504117d456a7e3c62641306c484fb07c50410d04851ce7c6264135fb2314b07c504110006324eac6264102608cc4af7c5041a9d91f

array([[0.02133932, 0.02008513, 0.02527952, ..., 0.05143411, 0.04330745,
        0.03488869],
       [0.02049939, 0.01899787, 0.02189651, ..., 0.05439872, 0.05720717,
        0.04802455],
       [0.01899452, 0.01730227, 0.01753171, ..., 0.05030817, 0.07519276,
        0.07538059],
       ...,
       [0.02265197, 0.02184701, 0.01708424, ..., 0.04455313, 0.06560078,
        0.05136503],
       [0.02095003, 0.01980173, 0.02072998, ..., 0.06371335, 0.07016074,
        0.0726635 ],
       [0.02093092, 0.02352743, 0.02272195, ..., 0.08081836, 0.07520615,
        0.07418419]], dtype=float32)

In [38]:
# Form a query for retrieving the sites that are within the circle
q = session.query(SiteData.geom).filter(gfunc.ST_Within(SiteData.geom, buffered_pit))

# Create a geopandas dataframe of SiteData geometry in the circle
nearby_pits = query_to_geopandas(q, engine)

In [ ]:
# Create a single plot to add everything to
fig, ax = plt.subplots(figsize=(8,8))

# Plot the DEM
img = show(dataset.read(1), ax=ax, transform=dataset.transform, cmap='terrain')

# Plot the contours of the DEM (Just for kicks!) at 10m intervals
show(dataset.read(1), contour=True, levels=[s for s in np.arange(3000, 4000, 10)], colors='dimgray', ax=ax, transform=dataset.transform)

# Plot the circle as blue with slight transparency
df_circle.plot(ax=ax, color='b', alpha=0.4, edgecolor='black')

# Plot the nearby pits as purple triangles
nearby_pits.plot(ax=ax, color='purple', marker='^', label='Nearby Pits',  edgecolor='black', markersize=150)

# Plot pit as a red triangle
df_pit.plot(ax=ax, color='r', marker='^', label=site_id,  edgecolor='black', markersize=150)

# Don't use scientific notation on the axis ticks
ax.ticklabel_format(style='plain', useOffset=False)

# Add x/y labels, a title, a legend and avoid overlapping labels
ax.set_xlabel('Easting [m]')
ax.set_ylabel('Northing [m]')
ax.set_title('Pit {} w/ {}m Radius Circle on USGS DEM'.format(site_id, buffer_dist))
ax.legend()
plt.tight_layout()

In [ ]:
# Close the dataset and end the database session
dataset.close()
session.close()